This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker import NNNodeBenchmarkerSSL
from graph_world.models.basic_gnn import GCN, SuperGAT
from torch_geometric.datasets import Planetoid, KarateClub, FakeDataset
from torch_geometric.transforms import RandomNodeSplit

from graph_world.self_supervised_learning.pretext_tasks.contrastive_based_different_scale import *


In [2]:
# Get dataset
dataset = FakeDataset(num_graphs = 1, avg_num_nodes = 100, num_channels = 16, num_classes = 4)[0]
dataset = RandomNodeSplit(split = "random", num_test = 20, num_val = 20, num_train_per_class=2)(dataset)

In [3]:
# Make the last node disconnected (for testing model robustness)
node = dataset.num_nodes - 1
mask = (dataset.edge_index[0] != node) & (dataset.edge_index[1] != node)
dataset.edge_index = dataset.edge_index[:, mask]

In [4]:
# Parameter setup (for cora)
benchmark_params = {
    'downstream_epochs' : 100,
    'pretext_epochs' : 100,
    'downstream_lr' : 3e-4,
    'pretext_lr' : 3e-4,
    'patience': 50
}

h_params = {
    'in_channels' : dataset.x.shape[1],
    'hidden_channels' : 128,
    'num_layers' : 2,
    'dropout' : 0.5,
}

pretext_params = {
    'alpha' : 0.15,
    'k' : 8
}

generator_config = {
    'num_clusters' : 4,
}

pretext_task = SUBGCON
training_scheme = 'JL'

In [5]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerSSL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params,
                pretext_task=pretext_task, pretext_params = pretext_params, training_scheme=training_scheme)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

([],
 [2.457749843597412,
  2.185495615005493,
  2.3042306900024414,
  2.1249043941497803,
  2.1014928817749023,
  2.178771495819092,
  2.1937310695648193,
  2.155031442642212,
  2.088836193084717,
  2.2277801036834717,
  2.316065788269043,
  2.2809855937957764,
  2.256649971008301,
  2.1206517219543457,
  2.2133941650390625,
  2.1477510929107666,
  2.0851807594299316,
  2.1042957305908203,
  2.0970468521118164,
  2.201967716217041,
  2.1924068927764893,
  2.123441219329834,
  2.091576337814331,
  2.1290788650512695,
  2.1613638401031494,
  2.044262409210205,
  2.067103862762451,
  1.9859774112701416,
  2.1571006774902344,
  2.1229281425476074,
  2.045536756515503,
  1.9637024402618408,
  1.9103254079818726,
  2.131218433380127,
  2.068760395050049,
  1.9914883375167847,
  1.8868788480758667,
  2.119051933288574,
  1.7593915462493896,
  2.0345709323883057,
  1.9444509744644165,
  2.1067464351654053,
  1.9399598836898804,
  1.8789503574371338,
  1.8475972414016724,
  1.897080659866333,
